# Práctica 05: Algoritmo Genético

## Juego de la vida con Algoritmo Genético 

Reglas:
- Nacimientos: Una célula muerta con exactamente tres vecinos vivos se convierte en una célula viva.
- Muerte uno: Una célula viva con uno o menos vecinos vivos muere.
- Muerte dos: Una célula viva con más de tres vecinos vivos muere, a menos que se  cumpla la condición especial.
- Condición Especial de Supervivencia: Si la población de cromosomas alcanza o  supera un $n$ especifica (80), las células vivas pueden soportar hasta cuatro  vecinos vivos sin morir.
- Supervivencia Normal: Si no se cumple la condición especial, una célula viva con dos o tres vecinos vivos sobrevive.
- Muerte tres: Si en $n$ generaciones no se alcanza el objetivo el juego termina. (80)

<center><h1>INICIO DEL CÓDIGO</h1></center>

In [1]:
import random

def inicializar_poblacion(tamaño_poblacion, tablero):
    """
    Crea una población inicial de cromosomas de manera aleatoria.
    
    Recibe:
    tamaño_poblacion (int): Número de cromosomas en la población.
    tablero (int): Tamaño de un lado del tablero del juego (número de células por lado).
    
    Devuelve:
    poblacion (list): Lista de cromosomas, donde cada cromosoma es una lista de bits.
    """
    # Creación de una población inicial de cromosomas para el Juego de la Vida
    poblacion = []

    # Generar cada cromosoma individualmente
    for _ in range(tamaño_poblacion):
        # Crear un cromosoma como una lista de células
        cromosoma = []

        # Generar cada célula en el cromosoma
        for _ in range(tablero**2):
            # Generar aleatoriamente el estado de la célula (viva o muerta)
            estado_celula = random.randint(0, 1)

            # Agregar el estado de la célula al cromosoma
            cromosoma.append(estado_celula)

    # Agregar el cromosoma a la población
    poblacion.append(cromosoma)
    return poblacion


def evaluar_aptitud(cromosoma, tablero, umbral_poblacion):
    """
    Evalúa la aptitud de un cromosoma según las reglas del juego.
    
    Recibe:
    cromosoma (list): Un cromosoma representando el estado del tablero.
    tablero (int): Tamaño de un lado del tablero del juego (número de células por lado).
    umbral_poblacion (int): Número específico de cromosomas que activa la condición especial.
    
    Devuelve:
    aptitud (float): Valor de aptitud del cromosoma.
    """
    # Convertir el cromosoma en una matriz 2D que representa el tablero    
    tablero = []

    # Generar cada fila del tablero
    for i in range(0, tablero**2, tablero):
        # Obtener una porción del cromosoma para representar una fila del tablero
        fila = cromosoma[i:i+tablero]

        # Agregar la fila al tablero
        tablero.append(fila)
    
    # Inicializar la aptitud a 0
    aptitud = 0
    
    # Aplicar las reglas del juego para determinar la aptitud
    for i in range(tablero):
        for j in range(tablero):
            vecinos_vivos = contar_vecinos_vivos(tablero, i, j)
            if tablero[i][j] == 1:  # Si la célula está viva
                if vecinos_vivos < 2 or vecinos_vivos > 3:
                    aptitud -= 1  # Penalizar la muerte 1 y 2
                else:
                    aptitud += 1  # Recompensar la supervivencia
            else:  # Si la célula está muerta
                if vecinos_vivos == 3:
                    aptitud += 1  # Recompensar el nacimiento de nuevas células
    
    # Verificar la condición especial
    if len(cromosoma) >= umbral_poblacion:
        aptitud *= 1.5  # Aumentar la aptitud si se cumple la condición especial
    
    return aptitud

# Función auxiliar para contar los vecinos vivos de una célula
def contar_vecinos_vivos(tablero, fila, columna):    
    pass
